In [1]:
%cd C:\Users\yukir\Documents\GitHub\Text_Mining\py

from nlp_processing import *
set_global_determinism(seed=1337)

%cd C:\Users\yukir\Documents\Monicas_workspace\Derma

C:\Users\yukir\Documents\GitHub\Text_Mining\py
True
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13116286800560252958
]
tf 2.5.3
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라


c:\Users\yukir\anaconda3\envs\NLP\lib\site-packages\scipy\io\matlab\mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from .mio5_utils import VarReader5


C:\Users\yukir\Documents\Monicas_workspace\Derma


In [35]:
import openpyxl
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models
from gensim.models import CoherenceModel
import gensim
import matplotlib.pyplot as plt

from gensim.models.ldamodel import LdaModel 
from gensim.models.callbacks import CoherenceMetric 
from gensim import corpora 
from gensim.models.callbacks import PerplexityMetric 
import logging 
import pickle 
import pyLDAvis

from gensim.models.coherencemodel import CoherenceModel 
import matplotlib.pyplot as plt

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

from konlpy.tag import Mecab 
from konlpy.tag import Okt
okt = Okt()
from tqdm import tqdm 
import re 
import pickle 
import csv


from hanspell import spell_checker

from nltk.corpus import stopwords  
stopwords.words('korean')[:10] 

stopwords_kor = stopwords.words('korean')

In [36]:
def clean_text(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    text = text.replace(".", " ").strip() 
    text = text.replace("·", " ").strip() 
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+' 
    text = re.sub(pattern=pattern, repl='', string=text) 
    return text 

def labels(x):
    i = str(x)
    i = i.replace('[','')
    i = i.replace(']','')
    i = i.replace(',','')
    k = i.split(' ')
    return k


In [63]:
data_servq = pd.read_excel("C:\\Users\\yukir\\Documents\\Monicas_workspace\\Derma_v2\\data\\★★[0512]derma_lables_v3.xlsx",index_col = 0)
data_serv = data_servq.copy()
reliability = labels(data_serv['keywords'][0])
responsiveness = labels(data_serv['keywords'][1])
assurance = labels(data_serv['keywords'][2])
empathy = labels(data_serv['keywords'][3])
tangible = labels(data_serv['keywords'][4])

In [5]:
data_org = pd.read_excel("C:\\Users\\yukir\\Documents\\Monicas_workspace\\Derma_v2\\Lbl2Vec\\data_derma\\Lbl2Vec_data\\★★[0503]data_for_Lvl2Vec.xlsx",index_col=0)
print(data_org.shape)
data_org.head()

(29363, 10)


,h,adr,reviews,stars,treatments,money,adr_else,dates,cleaned_reviews,stemmed
0,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),두피가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담해...,9.3,두피 가려움증,4900,57,2022-03-15,두피 가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담...,"['두피/Noun', '가려움증/Noun', '으로/Josa', '내원한/Verb'..."
1,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),대상포진으로 진료 받았어요 . 2주 이상 치료 받고 약 먹고 주사 맞으니까 진통이 ...,10.0,대상포진,4200,57,2021-12-20,대상포진으로 진료받았어요 2주 이상 치료받고 약 먹고 주사 맞으니까 진통이 가라앉았...,"['대상포진/Noun', '으로/Josa', '진료/Noun', '받았어요/Verb..."
2,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),여드름 고민이 있어서 방문했습니다피부 전문의라서 증상이 나타난 이유에 대해서 설명을...,10.0,관리 안 됨,4000,57,2021-08-11,여드름 고민이 있어서 방문했습니다 피부전문의라서 증상이 나타난 이유에 대해서 설명을...,"['여드름/Noun', '고민/Noun', '이/Josa', '있어서/Adjecti..."
3,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),문신제거문신한곳 일부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 천차...,10.0,문신제거,22000,57,2021-06-21,문신제거 문신한 곳일 부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 ...,"['문신/Noun', '제거/Noun', '문신/Noun', '한/Josa', '곳..."
4,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),원장님이 설명도 잘해주시고 병원도 엄청크고~(김해에서 유명한곳은 이유가있네요)!! ...,9.5,피부 트러블,4000,57,2021-06-07,원장님이설명도잘해주시고병원도엄청크고김해에서유명한곳은이유가있네요직원들도전부친절해서넘맘...,"['원장/Noun', '님/Suffix', '이/Determiner', '설명/No..."


In [6]:
def correction(x):
    try:
        x = x.replace('대기 시간', '대기시간')
        x = x.replace('점심 시간', '점심시간')
        x = x.replace('진료 시간', '진료시간')
        x = x.replace('치료 시간', '치료시간')
        x = x.replace('관리 시간', '관리시간')
    except:
        pass
    return x

In [7]:
data = data_org.copy()
data = data[['h', 'cleaned_reviews']]

data['cleaned_reviews'] = data['cleaned_reviews'].apply(correction)
# data['work'] = data['h']+'_'+data['cleaned_reviews']
data.head()

,h,cleaned_reviews
0,고운나라피부과의원,두피 가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담...
1,고운나라피부과의원,대상포진으로 진료받았어요 2주 이상 치료받고 약 먹고 주사 맞으니까 진통이 가라앉았...
2,고운나라피부과의원,여드름 고민이 있어서 방문했습니다 피부전문의라서 증상이 나타난 이유에 대해서 설명을...
3,고운나라피부과의원,문신제거 문신한 곳일 부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 ...
4,고운나라피부과의원,원장님이설명도잘해주시고병원도엄청크고김해에서유명한곳은이유가있네요직원들도전부친절해서넘맘...


In [8]:
def clean_text(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    text = text.replace(".", " ").strip() 
    text = text.replace("·", " ").strip() 
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|]+' 
    text = re.sub(pattern=pattern, repl='', string=text) 
    return text 

def stemming (text):
    morphs_list = []
    one_words = []
    result = []

    pos = okt.pos(text, join = False)
    # print(pos)
    try:
        for j in pos:
            if j.split('/')[1] == 'Noun':
                j = j.split('/')[0]
                morphs_list.append(j)
                        
            elif j.split('/')[1] =='Adjective':
                k = okt.morphs(j,  stem= True)
                k = k[0]
                morphs_list.append(k)
            elif j.split('/')[1] =='Verb':
                v = okt.morphs(j,  stem= True)
                v = v[0]
                morphs_list.append(v)

            elif j.split('/')[1] =='Adverb':
                ad = okt.morphs(j,  stem= True)
                v = v[0]
                morphs_list.append(ad)

        for i in morphs_list:
            if len(i) != 1:
                one_words.append(i)
        

        for i in one_words:
            if i not in stopwords_kor:
                result.append(i)
    except:
        result.append(text)
        pass

    return result

In [9]:
from hanspell import spell_checker

In [10]:
data['work'] = data['h']+'_'+data['cleaned_reviews']

In [14]:
len(results)

29203

In [ ]:
new_result = []

for i in results:
    new_result.append(i)

new_result[0]

In [18]:
reviews = pd.DataFrame({'hospital':hospital_list, 'reviews_pos':new_result})

In [19]:
reviews.to_csv('[0514]pos.csv')

In [20]:
reviews.to_excel('[0514]pos.xlsx')

In [26]:
hospital_list_v2=[]
sentences = []

for num, i in tqdm(enumerate(reviews.reviews_pos.tolist())):
    i = str(i)
    ss = i.split('Adjective')
    for i in ss:
        sentences.append(i)
        hos = reviews.hospital.tolist()[num]
        hospital_list_v2.append(hos)


29203it [01:27, 332.64it/s]


In [27]:
reviews_adj = pd.DataFrame({'hospital':hospital_list_v2, 'reviews_adj':sentences})

In [28]:
reviews_adj.head()

,hospital,reviews_adj
0,고운나라피부과의원,"['두피/Noun', '가려움증/Noun', '으로/Josa', '내원한/Verb'..."
1,고운나라피부과의원,"', '알려주시고/Verb', '참/Verb', '친절하셨어요/"
2,고운나라피부과의원,']
3,고운나라피부과의원,"['대상포진/Noun', '으로/Josa', '진료/Noun', '받았어요/Verb..."
4,고운나라피부과의원,"', '통증/Noun', '이/Josa', '가라앉아도/Verb', '후유증/Nou..."


In [31]:
print(reviews_adj.shape)
reviews_adj = reviews_adj.drop_duplicates()
print(reviews_adj.shape)

(242373, 2)
(228333, 2)


In [32]:
reviews_adj.to_excel('[0514]reviews_adj.xlsx')

In [3]:
import pandas as pd

In [25]:
reviews_adj = pd.read_excel("C:\\Users\\yukir\\Documents\\Monicas_workspace\\Derma\\[0514]reviews_adj.xlsx")

In [26]:
reviews_adj.shape

(228333, 3)

In [28]:
hospital_list_v3 = []
adj_verb = []

k=0
for num, i in tqdm(enumerate(reviews_adj.reviews_adj.tolist())):
    k+=1
    try:
        hos = reviews_adj.hospital.tolist()[num]
        if 'Verb' in i:
            i = i.split('Verb')
            for ii in i:
                adj_verb.append(ii)
                hospital_list_v3.append(hos)
        else:
            adj_verb.append(i)
            hospital_list_v3.append(hos)

        # print(228333 - k)
    except:
        pass
    # print(228333-k)

228333it [04:11, 907.17it/s]


In [29]:
reviews_adj_verb = pd.DataFrame({'hospital':hospital_list_v3, 'reviews_adj_verb':adj_verb})

In [30]:
reviews_adj_verb.head()

,hospital,reviews_adj_verb
0,고운나라피부과의원,"['두피/Noun', '가려움증/Noun', '으로/Josa', '내원한/"
1,고운나라피부과의원,"', '병원/Noun', '인데/Josa', '진료/Noun', '끝나고/"
2,고운나라피부과의원,"', '두피/Noun', '현미경/Noun', '사진/Noun', '을/Josa',..."
3,고운나라피부과의원,"', '상담/Noun', '해주셨어요/"
4,고운나라피부과의원,"', '제/Noun', '두피/Noun', '에/Josa', '맞는/"


In [31]:
reviews_adj_verb.to_csv('[0514]reviews_adj_verb.csv')

In [32]:
reviews_adj_verb.to_excel('[0514]reviews_adj_verb.xlsx')

In [33]:
print(reviews_adj_verb.shape)
reviews_adj_verb = reviews_adj_verb.drop_duplicates()
print(reviews_adj_verb.shape)

(552977, 2)
(514845, 2)


In [41]:
reviews_adj_verb = pd.read_excel('[0514]reviews_adj_verb.xlsx')
reviews_adj_verb.head()

,Unnamed: 0,hospital,reviews_adj_verb
0,0,고운나라피부과의원,"['두피/Noun', '가려움증/Noun', '으로/Josa', '내원한/"
1,1,고운나라피부과의원,"', '병원/Noun', '인데/Josa', '진료/Noun', '끝나고/"
2,2,고운나라피부과의원,"', '두피/Noun', '현미경/Noun', '사진/Noun', '을/Josa',..."
3,3,고운나라피부과의원,"', '상담/Noun', '해주셨어요/"
4,4,고운나라피부과의원,"', '제/Noun', '두피/Noun', '에/Josa', '맞는/"


In [45]:
reviews_adj_verb.reviews_adj_verb = reviews_adj_verb.reviews_adj_verb.apply(clean_text)
reviews_adj_verb.head()

,Unnamed: 0,hospital,reviews_adj_verb
0,0,고운나라피부과의원,두피 가려움증 으로 내원한
1,1,고운나라피부과의원,병원 인데 진료 끝나고
2,2,고운나라피부과의원,두피 현미경 사진 을 무료 로 찍어서
3,3,고운나라피부과의원,상담 해주셨어요
4,4,고운나라피부과의원,제 두피 에 맞는


In [43]:
def clean_text(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    text = text.replace(".", " ").strip() 
    text = text.replace("·", " ").strip() 
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|]+' 
    text = re.sub(pattern=pattern, repl='', string=text) 
    return text 

In [50]:
reviews_adj_verb_cleaned = reviews_adj_verb[['hospital', 'reviews_adj_verb']]
print(reviews_adj_verb_cleaned.shape)
reviews_adj_verb_cleaned = reviews_adj_verb_cleaned.drop_duplicates()
print(reviews_adj_verb_cleaned.shape)
reviews_adj_verb_cleaned = reviews_adj_verb_cleaned.dropna()
print(reviews_adj_verb_cleaned.shape)
reviews_adj_verb_cleaned.to_excel('[0514]reviews_adj_verb_cleaned_v2.xlsx')

(552977, 2)
(514158, 2)
(514158, 2)


In [51]:
def spell_check(x):
    x = spell_checker.check(x)
    x = x.checked
    x = x.strip()
    return x

In [47]:
reviews_adj_verb_cleaned.shape

(552977, 2)

In [53]:
reviews_adj_verb_cleaned.head()

,hospital,reviews_adj_verb
0,고운나라피부과의원,두피 가려움증 으로 내원한
1,고운나라피부과의원,병원 인데 진료 끝나고
2,고운나라피부과의원,두피 현미경 사진 을 무료 로 찍어서
3,고운나라피부과의원,상담 해주셨어요
4,고운나라피부과의원,제 두피 에 맞는


In [57]:
clean_reviews = []
hospital_list_v4 = []

for num, i in tqdm(enumerate(reviews_adj_verb_cleaned.reviews_adj_verb.tolist())):
    hos = reviews_adj_verb_cleaned.hospital.tolist()[num]
    try:
        i = spell_check(i)
        if len(i)>=1:
            clean_reviews.append(i)
            hospital_list_v4.append(hos)
    except:
        clean_reviews.append(i)
        hospital_list_v4.append(hos)





514158it [3:29:12, 40.96it/s]


In [58]:
reviews_adj_verb_cleaned= pd.DataFrame({'hospital':hospital_list_v4, 'reviews_adj_verb':clean_reviews})

In [59]:
reviews_adj_verb_cleaned.to_csv('[0514]reviews_adj_verb_cleaned.csv')

In [60]:
reviews_adj_verb_cleaned.to_csv('[0514]reviews_adj_verb_cleaned.csv')

In [61]:
reviews_adj_verb_cleaned.to_excel('[0514]reviews_adj_verb_cleaned.xlsx')

In [64]:
reliability = labels(data_serv['keywords'][0])
responsiveness = labels(data_serv['keywords'][1])
assurance = labels(data_serv['keywords'][2])
empathy = labels(data_serv['keywords'][3])
tangible = labels(data_serv['keywords'][4])

In [65]:
hospital_list_v5 = []
class_index = []
work_reviews=[]

for num, i in tqdm(enumerate(reviews_adj_verb_cleaned.reviews_adj_verb.tolist())):
    hos = reviews_adj_verb_cleaned.hospital.tolist()[num]
    
    try:
        for con in reliability:
            if con in i:
                hospital_list_v5.append(hos)
                work_reviews.append(i)
                class_index.append(1)

        for con in responsiveness:
            if con in i:
                hospital_list_v5.append(hos)
                work_reviews.append(i)
                class_index.append(2)

        for con in assurance:
            if con in i:
                hospital_list_v5.append(hos)
                work_reviews.append(i)
                class_index.append(3)

        for con in empathy:
            if con in i:
                hospital_list_v5.append(hos)
                work_reviews.append(i)
                class_index.append(4)

        for con in tangible:
            if con in i:
                hospital_list_v5.append(hos)
                work_reviews.append(i)
                class_index.append(5)

    except:
        pass




511776it [20:35, 414.24it/s]


In [66]:
class_df = pd.DataFrame({'class_index':class_index, 'hospital':hospital_list_v5, 'reviews':work_reviews})
print(class_df.shape)
class_df = class_df.drop_duplicates()
print(class_df.shape)
class_df = class_df.dropna()
print(class_df.shape)

(1363509, 3)
(707281, 3)
(707281, 3)


In [67]:
class_df.to_csv('[0514]class_df.csv')

In [68]:
class_df.to_excel('[0514]class_df.xlsx')

In [69]:
hospital_list_v5 = []
class_index = []
work_reviews=[]
keywords = []

for num, i in tqdm(enumerate(reviews_adj_verb_cleaned.reviews_adj_verb.tolist())):
    hos = reviews_adj_verb_cleaned.hospital.tolist()[num]
    
    try:
        for con in reliability:
            if con in i:
                hospital_list_v5.append(hos)
                work_reviews.append(i)
                class_index.append(1)
                keywords.append(con)

        for con in responsiveness:
            if con in i:
                hospital_list_v5.append(hos)
                work_reviews.append(i)
                class_index.append(2)
                keywords.append(con)

        for con in assurance:
            if con in i:
                hospital_list_v5.append(hos)
                work_reviews.append(i)
                class_index.append(3)
                keywords.append(con)

        for con in empathy:
            if con in i:
                hospital_list_v5.append(hos)
                work_reviews.append(i)
                class_index.append(4)
                keywords.append(con)

        for con in tangible:
            if con in i:
                hospital_list_v5.append(hos)
                work_reviews.append(i)
                class_index.append(5)
                keywords.append(con)

    except:
        pass




511776it [20:27, 417.02it/s]


In [71]:
class_df_v2 = pd.DataFrame({'class_index':class_index, 'keywords':keywords,'hospital':hospital_list_v5, 'reviews':work_reviews})
print(class_df_v2.shape)
class_df_v2 = class_df_v2.drop_duplicates()
print(class_df_v2.shape)
class_df_v2 = class_df_v2.dropna()
print(class_df_v2.shape)

(1363509, 4)
(843651, 4)
(843651, 4)


In [72]:
class_df_v2.head()

,class_index,keywords,hospital,reviews
0,3,,고운나라피부과의원,두피 가려움증으로 내원한
2,1,진료,고운나라피부과의원,병원인데 진료 끝나고
3,3,,고운나라피부과의원,병원인데 진료 끝나고
5,5,병원,고운나라피부과의원,병원인데 진료 끝나고
6,1,무료,고운나라피부과의원,두피 현미경 사진을 무료로 찍어서


In [74]:
class_df_v2.to_excel('[0514]class_df_v2.xlsx')

In [76]:
print(class_df_v2.shape)
class_df_v2['keywords'] = class_df_v2['keywords'].replace('','모지')
class_df_v2 = class_df_v2[class_df_v2['keywords']!='모지']
print(class_df_v2.shape)
class_df_v2 = class_df_v2.drop_duplicates()
print(class_df_v2.shape)
class_df_v2.head()

(843651, 4)
(332618, 4)
(332618, 4)


,class_index,keywords,hospital,reviews
2,1,진료,고운나라피부과의원,병원인데 진료 끝나고
5,5,병원,고운나라피부과의원,병원인데 진료 끝나고
6,1,무료,고운나라피부과의원,두피 현미경 사진을 무료로 찍어서
11,4,상담,고운나라피부과의원,상담해주셨어요
27,3,친절,고운나라피부과의원,친절하셨어요


In [77]:
class_df_v2.to_excel('[0514]class_df_v3.xlsx')